In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
root = "/content/drive/MyDrive/Thesis Final/Working Files/save model temp"
import os
from datasets import load_dataset

In [ ]:
st_chunk= 8
shard_chunk=15
num_shards = 20

In [ ]:
import json
path = "/content/drive/MyDrive/Thesis Final/Working Files/Dataset/"
news_file = path + "Dataset_4.json"

In [ ]:
news_list = []
with open(news_file, "r", encoding="utf-8") as fp:
  news_list = json.load(fp)

In [ ]:
from datasets import Dataset
all_contents = Dataset.from_list(news_list)

In [ ]:
all_contents
print(type(all_contents))

In [ ]:
shard_data = {i: list(range(i * len(all_contents) // num_shards, (i + 1) * len(all_contents) // num_shards)) for i in range(num_shards)}

In [ ]:
selected_chunk_data = all_contents.select(shard_data[shard_chunk])
print(f"Loaded chunk: {st_chunk}, shard: {shard_chunk}, total items: {len(selected_chunk_data)}")

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
tokenizer_checkpoint = os.path.join(root, "tokenizer")
model_checkpoint = "text_summarization_model"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(os.path.join(root, "pretrained model"))

In [ ]:
max_input_length, max_target_length = 512, 30

def preprocess_function(examples):
    inputs = tokenizer(examples["content"], max_length=max_input_length, truncation=True)
    labels = tokenizer(examples["title"], max_length=max_target_length, truncation=True)
    inputs["labels"] = labels["input_ids"]
    inputs["labels_mask"] = labels["attention_mask"]
    return inputs

In [ ]:
tokenized_datasets = selected_chunk_data.train_test_split(train_size=0.9, seed=20).map(preprocess_function, batched=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size, num_train_epochs = 3, 8

logging_steps = len(tokenized_datasets["train"]) // batch_size
training_args = Seq2SeqTrainingArguments(
    output_dir=os.path.join(root, "model"),
    evaluation_strategy="no",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    logging_steps=logging_steps,
    predict_with_generate=False,
)


In [ ]:
from transformers import DataCollatorForSeq2Seq
import gc

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
tokenized_datasets = tokenized_datasets.remove_columns(["train", "test"].column_names)
gc.collect()

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()
model.save_pretrained(os.path.join(root, "saved model"))

In [ ]:
## Halting
assert(False)

In [ ]:
from transformers import AutoModelForSeq2SeqLM
save_model=root+"/saved model"
model=AutoModelForSeq2SeqLM.from_pretrained(save_model)